In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import tensorflow as tf

model_path = "/content/drive/MyDrive/Colab Notebooks/waste_model_final_90.h5"
model = tf.keras.models.load_model(model_path)

print("✅ Model loaded successfully!")


✅ Model loaded successfully!


In [ ]:
pip install streamlit tensorflow pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 66.4 MB/s eta 0:00:00


In [ ]:


!pip install streamlit pyngrok tensorflow


In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("35JaYMZR8a8TWxowVkIKjD5xo3u_59swJmWeG6PXRNSxc8mxh")



In [ ]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
import numpy as np
from PIL import Image
from io import BytesIO
from reportlab.pdfgen import canvas
import matplotlib.pyplot as plt
import random
import pandas as pd
import os
from datetime import datetime
from gtts import gTTS
from streamlit_mic_recorder import mic_recorder
import requests
import base64

# ==================== PAGE CONFIG ====================
st.set_page_config(
    page_title="♻️ Ecosyne – The Green Guardian",
    page_icon="🌿",
    layout="wide",
)

# ==================== CUSTOM STYLE ====================
def custom_style():
    st.markdown("""
    <style>
        /* App Background */
        .stApp {
            background: linear-gradient(135deg, #d7f7e7, #a8e6cf, #20bd99);
            color: #003300;
            font-family: 'Poppins', sans-serif;
        }

        /* Main Titles */
        .main-title {
            text-align: center;
            font-size: 50px;
            font-weight: 900;
            color: #056b3a;
            margin-bottom: 5px;
            text-shadow: 1px 1px 5px rgba(0,0,0,0.2);
        }

        .sub-title {
            text-align: center;
            font-size: 18px;
            color: #2a7a4f;
            margin-bottom: 30px;
        }

        /* Sidebar */
        .sidebar .sidebar-content {
            background: linear-gradient(180deg, #00c853, #b2ff59);
            color: #003300;
        }

        .sidebar .sidebar-content .stRadio > div > label {
            color: #003300;
            font-weight: 600;
        }

        /* Buttons */
        .stButton>button {
            background-color: #00a86b;
            color: white;
            font-weight: 600;
            border-radius: 12px;
            padding: 0.6em 2em;
            border: none;
            transition: 0.3s;
        }

        .stButton>button:hover {
            background-color: #007f4d;
            transform: scale(1.05);
        }

        /* Progress Bars */
        .stProgress > div > div > div > div {
            background-color: #00a86b;
        }

        /* Chat bubbles */
        .chat-bubble-user {
            background-color: #00a86b;
            color: white;
            padding: 12px 18px;
            border-radius: 25px;
            margin: 6px 0;
            align-self: flex-end;
            max-width: 75%;
            box-shadow: 1px 1px 5px rgba(0,0,0,0.2);
        }

        .chat-bubble-bot {
            background-color: #f0f0f0;
            color: #003300;
            padding: 12px 18px;
            border-radius: 25px;
            margin: 6px 0;
            align-self: flex-start;
            max-width: 75%;
            box-shadow: 1px 1px 5px rgba(0,0,0,0.15);
        }

        /* Switch */
        .stSwitch > label > div[role="switch"][aria-checked="true"] {
            background-color: #ff7043 !important;
        }
        .stSwitch > label > div[role="switch"][aria-checked="false"] {
            background-color: #c8e6c9 !important;
        }

        /* PDF Download / File Uploader */
        .stFileUploader>div>div>input {
            border: 2px dashed #00a86b !important;
            border-radius: 10px;
            padding: 20px;
        }

        /* Divider */
        .stDivider {
            border-top: 2px solid #007f4d !important;
            margin: 25px 0 !important;
        }

        /* Footer */
        .stCaption {
            text-align: center;
            color: #004d26 !important;
            font-weight: 500;
        }

        /* Floating chat button */
        #ecobot-floating-btn {
            position: fixed;
            bottom: 30px;
            right: 30px;
            background-color: #00a86b;
            color: white;
            border: none;
            border-radius: 50%;
            width: 60px;
            height: 60px;
            font-size: 30px;
            cursor: pointer;
            box-shadow: 0 4px 8px rgba(0,0,0,0.3);
            z-index: 9999;
            display: flex;
            align-items: center;
            justify-content: center;
        }

        #ecobot-chat-window {
            position: fixed;
            bottom: 100px;
            right: 30px;
            width: 350px;
            max-height: 500px;
            background: #f0f8f0;
            border-radius: 15px;
            box-shadow: 0 8px 20px rgba(0,0,0,0.3);
            padding: 10px;
            display: none;
            flex-direction: column;
            z-index: 9999;
        }

        #ecobot-chat-window .header {
            background-color: #00a86b;
            color: white;
            font-weight: 600;
            padding: 10px;
            border-radius: 10px 10px 0 0;
            text-align: center;
        }

        #ecobot-chat-window .messages {
            flex: 1;
            overflow-y: auto;
            padding: 5px;
            margin-bottom: 5px;
        }

        #ecobot-chat-window input {
            width: 100%;
            padding: 8px;
            border-radius: 8px;
            border: 1px solid #ccc;
        }
    </style>
    """, unsafe_allow_html=True)

custom_style()

# ==================== MODEL ====================
@st.cache_resource
def load_model():
    model_path = "/content/drive/MyDrive/Colab Notebooks/waste_model_final_90.h5"
    return tf.keras.models.load_model(model_path)

try:
    model = load_model()
    st.sidebar.success("✅ Ecosyne Model Loaded")
except Exception as e:
    st.sidebar.error("❌ Model not found – please verify the file path.")
    model = None

# ==================== CLASS LABELS ====================
CLASS_NAMES = ['aerosol_cans', 'aluminum_food_cans', 'aluminum_soda_cans', 'cardboard_boxes', 'cardboard_packaging', 'clothing', 'coffee_grounds', 'disposable_plastic_cutlery', 'eggshells', 'food_waste', 'glass_beverage_bottles', 'glass_cosmetic_containers', 'glass_food_jars', 'images', 'magazines', 'newspaper', 'office_paper', 'paper_cups', 'plastic_cup_lids', 'plastic_detergent_bottles', 'plastic_food_containers', 'plastic_shopping_bags', 'plastic_soda_bottles', 'plastic_straws', 'plastic_trash_bags', 'plastic_water_bottles', 'shoes', 'steel_food_cans', 'styrofoam_cups', 'styrofoam_food_containers', 'tea_bags']

# ==================== TITLE ====================
st.markdown("<h1 class='main-title'>🌿 Ecosyne – The Green Guardian</h1>", unsafe_allow_html=True)
st.markdown("<p class='sub-title'>Smarter Waste Sorting through AI + Play + Sustainability ♻️</p>", unsafe_allow_html=True)

# ==================== SIDEBAR NAVIGATION ====================
st.sidebar.header("🌍 Navigation")
page = st.sidebar.radio("Go to:", ["🏠 Home", "💬 EcoBot", "💡 About", "👩‍💻 Developer"])

# ==================== ECO TRACKER ====================
eco_log_path = "/content/eco_behavior_log.csv"

def log_prediction(pred_class, category):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    record = pd.DataFrame([[timestamp, pred_class, category]], columns=["timestamp", "predicted_class", "category"])
    if os.path.exists(eco_log_path):
        df_old = pd.read_csv(eco_log_path)
        df_new = pd.concat([df_old, record], ignore_index=True)
    else:
        df_new = record
    df_new.to_csv(eco_log_path, index=False)

def show_eco_tracker():
    if os.path.exists(eco_log_path):
        df = pd.read_csv(eco_log_path)
        st.subheader("🌱 Your Eco Behavior Tracker")
        total = len(df)
        recyclable = len(df[df["category"] == "Recyclable"])
        non_recyclable = len(df[df["category"] == "Non-Recyclable"])
        st.write(f"**Total Items Classified:** {total}")
        st.progress(min(total / 50, 1.0))
        st.caption(f"🎯 {total}/50 classifications this week")
        fig, ax = plt.subplots()
        ax.pie([recyclable, non_recyclable], labels=["Recyclable", "Non-Recyclable"], colors=["#00a86b", "#ff6363"], autopct='%1.1f%%', startangle=90)
        ax.axis("equal")
        st.pyplot(fig)
    else:
        st.info("Start classifying waste to see your eco progress here 🌿")

# ==================== PAGE 1: WASTE CLASSIFIER ====================
if page == "🏠 Home":
    uploaded_file = st.file_uploader("📸 Upload a Waste Image to Classify", type=["jpg", "jpeg", "png"])
    if uploaded_file and model is not None:
        image = Image.open(uploaded_file).convert('RGB')
        st.image(image, caption="🖼️ Uploaded Image", use_container_width=True)
        with st.spinner("🤖 AI analyzing your image..."):
            img = image.resize((224, 224))
            img_array = np.expand_dims(np.array(img) / 255.0, axis=0)
            preds = model.predict(img_array)
            predicted_index = int(np.argmax(preds))
            confidence = float(np.max(preds)) * 100
            predicted_class = CLASS_NAMES[predicted_index].replace("_", " ").title()
        st.success(f"🏷️ Predicted Waste Type: **{predicted_class}**")
        st.progress(int(confidence))
        st.info(f"🌱 Confidence Level: {confidence:.2f}%")
        st.subheader("📊 AI Confidence Analysis")
        fig, ax = plt.subplots(figsize=(5, 5))
        ax.pie([confidence, 100 - confidence], labels=["Confidence", "Uncertainty"], autopct='%1.1f%%', colors=["#00a86b", "#393e46"], startangle=90)
        ax.axis("equal")
        st.pyplot(fig)
        recyclable = ["aluminum_food_cans", "aluminum_soda_cans", "cardboard_boxes", "cardboard_packaging",
                      "glass_beverage_bottles", "glass_food_jars", "newspaper", "office_paper",
                      "steel_food_cans", "plastic_water_bottles"]
        category = "Recyclable" if CLASS_NAMES[predicted_index] in recyclable else "Non-Recyclable"
        st.markdown(f"### ♻️ This item belongs to: **{category} Waste Bin**")
        if category == "Recyclable": st.balloons()
        else: st.snow()
        log_prediction(predicted_class, category)
        st.divider()
        show_eco_tracker()
        st.subheader("📄 Download Smart Report")
        def generate_pdf(pred_class, conf, category):
            buffer = BytesIO()
            c = canvas.Canvas(buffer)
            c.setFont("Helvetica-Bold", 20)
            c.drawString(100, 780, "♻️ Ecosyne Waste Report")
            c.setFont("Helvetica", 14)
            c.drawString(100, 740, f"Waste Type: {pred_class}")
            c.drawString(100, 720, f"Confidence: {conf:.2f}%")
            c.drawString(100, 700, f"Category: {category}")
            c.drawString(100, 680, "Thank you for making the planet cleaner 🌎")
            c.save()
            buffer.seek(0)
            return buffer
        pdf_buffer = generate_pdf(predicted_class, confidence, category)
        st.download_button("📥 Download Report (PDF)", data=pdf_buffer, file_name="Ecosyne_Report.pdf", mime="application/pdf")

# ==================== PAGE 2: ECOBOT ====================
elif page == "💬 EcoBot":
    st.subheader("🤖 EcoBot – Your Green Assistant")
    st.markdown("<h5 style='color:#056b3a;'>🎧 Eco Voice Mode</h5>", unsafe_allow_html=True)
    voice_mode = st.checkbox("Enable Eco Voice 🎤")
    user_input = ""
    if voice_mode:
        st.info("🎙️ Speak your question and let EcoBot reply with voice 🌿")
        audio_text = mic_recorder(start_prompt="🎙️ Start Recording", stop_prompt="🛑 Stop Recording", just_once=True)
        if audio_text:
            try:
                import speech_recognition as sr
                from pydub import AudioSegment
                from io import BytesIO
                recognizer = sr.Recognizer()
                audio_bytes = BytesIO(audio_text["bytes"])
                try:
                    sound = AudioSegment.from_file(audio_bytes, format="webm")
                except:
                    audio_bytes.seek(0)
                    sound = AudioSegment.from_file(audio_bytes, format="wav")
                wav_fp = BytesIO()
                sound.export(wav_fp, format="wav")
                wav_fp.seek(0)
                with sr.AudioFile(wav_fp) as source:
                    audio_data = recognizer.record(source)
                    user_input = recognizer.recognize_google(audio_data)
                    st.success("✅ Voice recognized!")
            except sr.UnknownValueError:
                st.warning("❌ Could not understand your voice — please try again.")
            except sr.RequestError:
                st.error("🌐 Network issue: Google API unreachable.")
            except Exception as e:
                st.error(f"⚠️ Unexpected error: {e}")
    if not user_input:
        user_input = st.text_input("💬 Or type your question:")
    if user_input:
        st.markdown(f"<div class='chat-bubble-user'>{user_input}</div>", unsafe_allow_html=True)
        if "recycle" in user_input.lower():
            eco_reply = "Recycling helps reduce pollution and conserve natural resources."
        elif "plastic" in user_input.lower():
            eco_reply = "Plastic waste should be separated and recycled whenever possible."
        elif "save water" in user_input.lower():
            eco_reply = "Turn off taps when not in use and collect rainwater for reuse!"
        else:
            eco_reply = "Let's keep the planet green and clean! 🌏"
        st.markdown(f"<div class='chat-bubble-bot'>{eco_reply}</div>", unsafe_allow_html=True)
        try:
            tts = gTTS(text=eco_reply, lang='en')
            tts.save("eco_voice.mp3")
            with open("eco_voice.mp3", "rb") as f:
                audio_bytes = f.read()
            b64 = base64.b64encode(audio_bytes).decode()
            st.markdown(f"""
                <audio controls autoplay>
                    <source src="data:audio/mp3;base64,{b64}" type="audio/mp3">
                </audio>
            """, unsafe_allow_html=True)
        except Exception as e:
            st.warning(f"Voice playback error: {e}")

# ==================== ABOUT & DEVELOPER ====================
elif page == "💡 About":
    st.subheader("💚 About Ecosyne")
    st.write("""
        **Ecosyne** merges AI with sustainability.
        It classifies waste, promotes recycling, and educates users interactively.
        Every scan brings us closer to a cleaner planet 🌏.
    """)

elif page == "👩‍💻 Developer":
    st.subheader("👩‍💻 About the Developer")
    st.write("""
        Developed by **Dhivashini R**
        Passionate about Artificial Intelligence and Environmental Sustainability 💚
    """)
    st.markdown("""
        <a href="https://www.linkedin.com/in/dhivashini-r" target="_blank">
            <button style='background-color:#00a86b; color:white; padding:10px 20px; border:none; border-radius:10px; font-size:16px;'>
                💼 Connect on LinkedIn
            </button>
        </a>
    """, unsafe_allow_html=True)

# ==================== FOOTER ====================
st.divider()
st.caption("🚀 Ecosyne AI | Designed with 💚 by Dhivashini | © 2025")



Overwriting app.py


In [1]:
pip install nbmerge


  Preparing metadata (setup.py) ... done
  Created wheel for nbmerge: filename=nbmerge-0.0.4-py2.py3-none-any.whl size=6396 sha256=dbdac1c4ca554ab699f9e07989e8cfcaf60f5dae5edff21e37bc7ae1e0c129a7
  Stored in directory: /root/.cache/pip/wheels/66/ec/e3/e3e9f9c3007f764582fcd1d93e2c9d296355985422f62c55f1
Successfully built nbmerge


In [4]:

!nbmerge week2.ipynb app.ipynb > finalproject.ipynb


Traceback (most recent call last):
  File "/usr/local/bin/nbmerge", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/nbmerge/__init__.py", line 189, in main
    plan = parse_plan(args)
           ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/nbmerge/__init__.py", line 173, in parse_plan
    raise IOError("Notebook `{}` does not exist".format(file_path))
OSError: Notebook `week2.ipynb` does not exist


In [5]:

!ls



finalproject.ipynb  sample_data


In [ ]:
from google.colab import files
files.upload()


In [ ]:
from flask import Flask, request, jsonify, send_file
from flask_cors import CORS
import speech_recognition as sr
from gtts import gTTS
import os
import pickle
import numpy as np
import io
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import json

app = Flask(__name__)
CORS(app)

# =============================
# 1️⃣ Intent Model Setup
# =============================

MODEL_PATH = "eco_intent_model.pkl"
RESPONSES_PATH = "responses.json"

# Sample data for eco-intent model
training_data = {
    "hi": "greet",
    "hello": "greet",
    "hey": "greet",
    "how to recycle plastic": "plastic",
    "what to do with bottles": "plastic",
    "can i recycle glass": "glass",
    "dispose paper waste": "paper",
    "where to throw banana peel": "food",
    "how to reduce waste": "advice",
    "tell me eco tip": "advice"
}

# Corresponding replies
responses = {
    "greet": "Hello there! I’m EcoBot, your recycling assistant. How can I help?",
    "plastic": "Plastic bottles and containers go in the recycling bin after rinsing.",
    "glass": "Glass jars and bottles are recyclable — make sure they’re clean and dry.",
    "paper": "Paper and cardboard should be kept dry and placed in the blue bin.",
    "food": "Organic waste like banana peels goes into compost. 🌱",
    "advice": "Reduce, Reuse, Recycle — even small changes make a big impact!"
}

# Save responses.json
if not os.path.exists(RESPONSES_PATH):
    with open(RESPONSES_PATH, "w") as f:
        json.dump(responses, f)

# Train model if not present
if not os.path.exists(MODEL_PATH):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(list(training_data.keys()))
    y = list(training_data.values())

    model = MultinomialNB()
    model.fit(X, y)

    with open(MODEL_PATH, "wb") as f:
        pickle.dump((vectorizer, model), f)

# Load model
with open(MODEL_PATH, "rb") as f:
    vectorizer, model = pickle.load(f)

with open(RESPONSES_PATH, "r") as f:
    responses = json.load(f)

# =============================
# 2️⃣ API: Text → Reply
# =============================
@app.route("/text_chat", methods=["POST"])
def text_chat():
    data = request.json
    text = data.get("text", "").lower().strip()
    if not text:
        return jsonify({"error": "Empty text"}), 400

    X = vectorizer.transform([text])
    intent = model.predict(X)[0]
    reply = responses.get(intent, "Sorry, I’m still learning about that topic.")

    # Generate speech
    tts = gTTS(reply)
    audio_fp = io.BytesIO()
    tts.write_to_fp(audio_fp)
    audio_fp.seek(0)

    return send_file(audio_fp, mimetype="audio/mpeg")

# =============================
# 3️⃣ API: Voice → Text → Reply
# =============================
@app.route("/voice_chat", methods=["POST"])
def voice_chat():
    if "audio" not in request.files:
        return jsonify({"error": "No audio file received"}), 400

    audio_file = request.files["audio"]
    recognizer = sr.Recognizer()

    with sr.AudioFile(audio_file) as source:
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio_data)
        except Exception:
            return jsonify({"error": "Could not recognize speech"}), 500

    X = vectorizer.transform([text])
    intent = model.predict(X)[0]
    reply = responses.get(intent, "Sorry, I’m still learning about that topic.")

    # Convert reply to audio
    tts = gTTS(reply)
    audio_fp = io.BytesIO()
    tts.write_to_fp(audio_fp)
    audio_fp.seek(0)

    return send_file(audio_fp, mimetype="audio/mpeg")

# =============================
# Run server
# =============================
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
!pip install flask flask-cors gtts SpeechRecognition scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 7.2 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.3.0
    Uninstalling click-8.3.0:
      Successfully uninstalled click-8.3.0


In [ ]:
from pyngrok import ngrok
import threading, time

# Run Streamlit app
def run_streamlit():
    !streamlit run app.py --server.port 8501

thread = threading.Thread(target=run_streamlit)
thread.start()

# Wait for app to start
time.sleep(15)

# Create ngrok tunnel
public_url = ngrok.connect(8501)
print("🌍 Your Streamlit app is live here:")
print(public_url.public_url)





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.138.253.145:8501

🌍 Your Streamlit app is live here:
https://osmotically-preearthly-royal.ngrok-free.dev


In [ ]:
!pip install SpeechRecognition pydub


In [ ]:
!apt install ffmpeg -y


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [ ]:
pip install streamlit-mic-recorder SpeechRecognition pydub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 26.3 MB/s eta 0:00:00


In [ ]:
pip install gTTS streamlit-mic-recorder


In [ ]:
pip install streamlit-webrtc SpeechRecognition pyaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 1.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.2/220.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 kB 4.3 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for pyaudio (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a pro

In [ ]:
from gtts import gTTS
from IPython.display import Audio

text = "Hello, this is Ecosyne speaking. Let's make the world greener!"
tts = gTTS(text, lang='en', tld='co.in')
tts.save("eco_test.mp3")
Audio("eco_test.mp3")


In [ ]:
pip install streamlit-mic-recorder gtts


In [ ]:
!ls /content/drive/MyDrive/Colab\ Notebooks/


ls: cannot access '/content/drive/MyDrive/Colab Notebooks/': No such file or directory


In [ ]:
import os
test_dir = '/content/drive/MyDrive/Colab Notebooks/waste_data_split/test'
class_names = sorted(os.listdir(test_dir))
print(class_names)


['aerosol_cans', 'aluminum_food_cans', 'aluminum_soda_cans', 'cardboard_boxes', 'cardboard_packaging', 'clothing', 'coffee_grounds', 'disposable_plastic_cutlery', 'eggshells', 'food_waste', 'glass_beverage_bottles', 'glass_cosmetic_containers', 'glass_food_jars', 'images', 'magazines', 'newspaper', 'office_paper', 'paper_cups', 'plastic_cup_lids', 'plastic_detergent_bottles', 'plastic_food_containers', 'plastic_shopping_bags', 'plastic_soda_bottles', 'plastic_straws', 'plastic_trash_bags', 'plastic_water_bottles', 'shoes', 'steel_food_cans', 'styrofoam_cups', 'styrofoam_food_containers', 'tea_bags']


In [ ]:
from pyngrok import ngrok
ngrok.kill()  # closes all tunnels
!streamlit run app.py &>/dev/null&
public_url = ngrok.connect(8501)
print(public_url)


NgrokTunnel: "https://osmotically-preearthly-royal.ngrok-free.dev" -> "http://localhost:8501"


In [ ]:
import tensorflow as tf
model = tf.keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/waste_model_final_90.h5")
print("Model output shape:", model.output_shape)


Model output shape: (None, 31)


In [ ]:
!pip install reportlab


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.3 MB/s eta 0:00:00
